# Optimización de Pesos de Evaluación (Versión Definitiva)

Restricciones:
1. **Granularidad**: Módulos divididos en Quices y Tareas.
2. **Regla de Quices**: Quices < 30% del módulo.
3. **Énfasis en Módulo 2**: $W_{M2} = 1.1 \times W_{M1}$ (M2 es un 10% mayor).
4. **Concepto Profesor**: Nota discreta (2.0, 2.5, ..., 5.0) basada en rendimiento previo, peso **<= 15%**.
5. **Reglas Globales**:
   - Examen Final >= 20% y **<= 40%**
   - Tarea Individual <= 20%
   - Zona Gris: Penalizamos el rango [2.90, 3.10].

## 1. Configuración Inicial

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

np.random.seed(42)

## 2. Generación de Escenarios (Distribuciones STEM)

Usamos generadores más realistas:
- Dist. Bimodal (para clases 'filtro').
- Dist. Sesgada Negativa (para tareas donde la mayoría aprueba).
- Dist. Normal (casos estándar).

In [2]:
def generar_distribucion_bimodal(n, low_center=1.5, high_center=4.5, ratio=0.5, sigma=0.6):
    n_high = int(n * ratio)
    n_low = n - n_high
    high_grades = np.random.normal(high_center, sigma, n_high)
    low_grades = np.random.normal(low_center, sigma, n_low)
    combined = np.concatenate([high_grades, low_grades])
    return np.clip(combined, 0, 5)

def generar_distribucion_negative_skew(n, mode=4.2, sigma=1.0):
    raw = mode - np.random.exponential(scale=0.8, size=n)
    return np.clip(raw, 0, 5)

def generar_datos_cientificos(n_estudiantes=2000):
    data = {}
    
    # MÓDULO 1: Fácil (Intro)
    data['M1_Q'] = generar_distribucion_negative_skew(n_estudiantes, mode=4.5)
    data['M1_T'] = generar_distribucion_negative_skew(n_estudiantes, mode=4.2)
    
    # MÓDULO 2: Filtro (Bimodal)
    data['M2_Q'] = generar_distribucion_bimodal(n_estudiantes, low_center=1.2, high_center=4.0, ratio=0.4)
    data['M2_T'] = generar_distribucion_bimodal(n_estudiantes, low_center=2.0, high_center=4.5, ratio=0.5) 
    
    # MÓDULO 3: Avanzado (Normal)
    data['M3_Q'] = np.clip(np.random.normal(3.0, 1.2, n_estudiantes), 0, 5)
    data['M3_T'] = np.clip(np.random.normal(3.5, 1.0, n_estudiantes), 0, 5)
    
    # EXAMEN FINAL
    data['Examen_Final'] = np.clip(np.random.normal(2.8, 1.1, n_estudiantes), 0, 5)
    
    df = pd.DataFrame(data)
    
    # (NOTA CONCEPTO ELIMINADA)
    
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    return df.round(2)

df_notas = generar_datos_cientificos(2000)
# Features reducido a 7
features = ['M1_Q', 'M1_T', 'M2_Q', 'M2_T', 'M3_Q', 'M3_T', 'Examen_Final']
display(df_notas.describe())
df_notas.head()

,M1_Q,M1_T,M2_Q,M2_T,M3_Q,M3_T,Examen_Final
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.00000
mean,3.699445,3.417365,2.308250,3.190665,3.008075,3.44423,2.80637
std,0.795124,0.760018,1.490038,1.340696,1.148524,0.92744,1.06758
min,0.000000,0.000000,0.000000,0.070000,0.000000,0.00000,0.00000
25%,3.390000,3.097500,1.070000,1.970000,2.230000,2.81000,2.10000
50%,3.930000,3.660000,1.735000,3.230000,3.010000,3.47000,2.80000
75%,4.280000,3.970000,3.792500,4.470000,3.850000,4.12000,3.53000
max,4.500000,4.200000,5.000000,5.000000,5.000000,5.00000,5.00000


,M1_Q,M1_T,M2_Q,M2_T,M3_Q,M3_T,Examen_Final
0,3.30,2.85,1.29,2.77,2.77,2.17,2.42
1,4.36,2.54,3.47,4.94,5.00,2.48,2.90
2,0.01,4.03,0.63,1.46,1.83,3.58,2.94
3,2.71,4.16,1.39,5.00,4.19,5.00,3.04
4,4.43,4.02,1.26,1.72,5.00,2.45,1.49


## 3. El "Oráculo" (Criterio Humano NUEVO)

Nuevas reglas centradas en el Examen Final:
1. **Muerte Súbita**: Si `Final < 2.5` -> **PIERDE** (sin importar nada más).
2. **Debilidad**: Si `Final < 3.0` Y `Promedio Módulos < 3.0` -> **PIERDE**.
3. **Redención**: Si `2.5 <= Final < 3.0` PERO `Todos los Módulos > 3.5` -> **GANA**.
4. En otros casos, mantenemos el criterio base (M2 es filtro, promedio simple debe aprobar).

In [3]:
def get_oracle_decisions_vectorized(weights, df):
    # Weights: [m1q, m1t, m2q, m2t, m3q, m3t, final]
    
    eps = 1e-9
    w_m1_total = weights[0] + weights[1] + eps
    w_m2_total = weights[2] + weights[3] + eps
    w_m3_total = weights[4] + weights[5] + eps
    
    n_m1 = (df['M1_Q'] * weights[0] + df['M1_T'] * weights[1]) / w_m1_total
    n_m2 = (df['M2_Q'] * weights[2] + df['M2_T'] * weights[3]) / w_m2_total
    n_m3 = (df['M3_Q'] * weights[4] + df['M3_T'] * weights[5]) / w_m3_total
    
    final = df['Examen_Final']
    
    # Avg Módulos (Simple average of the 3 modules as per previous logic)
    avg_modulos = (n_m1 + n_m2 + n_m3) / 3.0
    
    # Initialize Decisions (Default = 1/Pass)
    decisions = np.ones(len(df), dtype=int)
    
    # 1. MUERTE SÚBITA
    mask_sudden_death = (final < 2.5)
    decisions[mask_sudden_death] = 0
    
    # 2. DEBILIDAD
    mask_weakness = (final < 3.0) & (avg_modulos < 3.0)
    decisions[mask_weakness] = 0
    
    # 3. Old Filter (M2 < 2.5)
    mask_m2_fail = (n_m2 < 2.5)
    decisions[mask_m2_fail] = 0
    
    # 4. Global Mean Check (Raw Mean of columns for stability/human consistency)
    raw_cols = ['M1_Q', 'M1_T', 'M2_Q', 'M2_T', 'M3_Q', 'M3_T', 'Examen_Final']
    promedio_total = df[raw_cols].mean(axis=1)
    decisions[promedio_total < 3.0] = 0
    
    # 5. REDENCIÓN (Can override checks)
    mask_redemption = (final >= 2.5) & (final < 3.0) & (n_m1 > 3.5) & (n_m2 > 3.5) & (n_m3 > 3.5)
    decisions[mask_redemption] = 1
    
    return decisions


## 4. Optimización Directa

Optimización con pesos restringidos (Concepto <= 15%).

In [4]:
def g_threshold_vectorized(X, U):
    return np.clip(X / U, 0, 1)

def calcular_nota_interpolacion_vectorizada(weights_7, df, U=3):
    # weights_7: [m1q, m1t, m2q, m2t, m3q, m3t, final]
    
    w_m1_local = weights_7[0] + weights_7[1]
    w_m2_local = weights_7[2] + weights_7[3]
    w_m3_local = weights_7[4] + weights_7[5]
    w_final = weights_7[6]
    
    eps = 1e-9
    
    # Notas Módulos
    n_m1 = (df['M1_Q']*weights_7[0] + df['M1_T']*weights_7[1]) / (w_m1_local + eps)
    n_m2 = (df['M2_Q']*weights_7[2] + df['M2_T']*weights_7[3]) / (w_m2_local + eps)
    n_m3 = (df['M3_Q']*weights_7[4] + df['M3_T']*weights_7[5]) / (w_m3_local + eps)
    n_fin = df['Examen_Final']
    
    # Matriz X
    X_comps = np.stack([n_m1, n_m2, n_m3, n_fin], axis=1)
    W_global = np.array([w_m1_local, w_m2_local, w_m3_local, w_final])
    
    # 1. Promedio Clásico (A)
    A = np.sum(X_comps * W_global, axis=1)
    
    # 2. Piso (m)
    m_floor = np.min(X_comps[:, :3], axis=1)
    
    # 3. Factores g(x)
    G = g_threshold_vectorized(X_comps, U)
    
    # 4. Pesos Locales U_i
    U_factors = np.zeros_like(G)
    for i in range(4):
        others = [j for j in range(4) if j != i]
        U_factors[:, i] = np.prod(G[:, others], axis=1)
        
    # 5. Suma Local Penalizada S_local
    S_local = np.sum(X_comps * W_global * U_factors, axis=1)
    
    # 6. Factor Local F_local
    F_local = np.zeros_like(A)
    mask_A = A > eps
    F_local[mask_A] = np.clip(S_local[mask_A] / A[mask_A], 0, 1)
    F_local[~mask_A] = 1.0
    
    # 7. Nota Final Interpolada (Raw)
    N_raw = m_floor + (A - m_floor) * F_local
    N_raw = np.maximum(N_raw, m_floor)
    
    # --- AJUSTES DE USUARIO ---
    # 1. Evitar aumento de nota (Conservative) -> N <= A
    # 2. Si ya perdió (A < 3.0), no disminuir más -> N >= A
    # Conclusión: Si A < 3.0, N=A. Si A >= 3.0, N = min(N_raw, A)
    
    # Nota: Usamos 2.95 como umbral de redondeo seguro o 3.0 estricto
    threshold_fail = 2.95
    
    N_final = np.where(
        A < threshold_fail,
        A,                       # Si reprueba por promedio, MANTIENE promedio (no castigar más)
        np.minimum(N_raw, A)     # Si aprueba, aplica penalización (N_raw) pero nunca bonificación (min)
    )
    
    return N_final, A, S_local, F_local, m_floor

def get_final_scores(weights, df):
    N, _, _, _, _ = calcular_nota_interpolacion_vectorizada(weights, df)
    return N

In [5]:
def funcion_objetivo(weights, X_ignored=None, y_ignored=None):
    # X_ignored, y_ignored are kept for signature compatibility
    
    # 1. Generar el "Ground Truth" Dinámico
    y_true_dynamic = get_oracle_decisions_vectorized(weights, df_notas)
    
    # 2. Calcular Notas Propuestas
    notas_finales = get_final_scores(weights, df_notas)
    
    passing_grade = 2.95
    
    pass_mask = (y_true_dynamic == 1)
    fail_mask = (y_true_dynamic == 0)
    
    # 3. Calcular Error
    err_pass = np.maximum(0, passing_grade - notas_finales[pass_mask])**2
    err_fail = np.maximum(0, notas_finales[fail_mask] - passing_grade)**2
    
    return np.sum(err_pass) + np.sum(err_fail)

# --- Restoring Constraints & Initialization ---
cons = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
cons.append({'type': 'ineq', 'fun': lambda w: 0.3*w[1] - 0.7*w[0]}) # Q < 30% M1
cons.append({'type': 'ineq', 'fun': lambda w: 0.3*w[3] - 0.7*w[2]}) # Q < 30% M2
cons.append({'type': 'ineq', 'fun': lambda w: 0.3*w[5] - 0.7*w[4]}) # Q < 30% M3
cons.append({'type': 'ineq', 'fun': lambda w: w[6] - 0.20}) # Final >= 20
cons.append({'type': 'ineq', 'fun': lambda w: 0.40 - w[6]}) # Final <= 40
cons.append({'type': 'ineq', 'fun': lambda w: 0.20 - w[1]}) # Tarea <= 20
cons.append({'type': 'ineq', 'fun': lambda w: 0.20 - w[3]})
cons.append({'type': 'ineq', 'fun': lambda w: 0.20 - w[5]})
cons.append({'type': 'eq', 'fun': lambda w: (w[2] + w[3]) - 1.1 * (w[0] + w[1])}) # M2 = 1.1 M1
cons.append({'type': 'eq', 'fun': lambda w: (w[0] + w[1]) - (w[4] + w[5])})       # M1 = M3

bnds = [(0.01, 1.0)] * 7
w0 = np.ones(7) / 7

print("Optimizando con Oráculo Dinámico...")
# Ajustamos args a vacio
res = minimize(funcion_objetivo, w0, args=(None, None), constraints=cons, bounds=bnds)
w_best = res.x

# --- REPORTE DE PESOS ÓPTIMOS ---
print("\n=== PESOS ÓPTIMOS (INTERPOLACIÓN) ===")
# w_best: [m1q, m1t, m2q, m2t, m3q, m3t, final]
w = w_best
m1_total = w[0] + w[1]
m2_total = w[2] + w[3]
m3_total = w[4] + w[5]
final = w[6]

print(f"Modulo 1 (Q+T): {m1_total:.1%} (Q:{w[0]:.1%}, T:{w[1]:.1%})")
print(f"Modulo 2 (Q+T): {m2_total:.1%} (Q:{w[2]:.1%}, T:{w[3]:.1%})")
print(f"Modulo 3 (Q+T): {m3_total:.1%} (Q:{w[4]:.1%}, T:{w[5]:.1%})")
print(f"Examen Final  : {final:.1%}")

# Validar Exactitud con Oráculo
y_pred_model = get_oracle_decisions_vectorized(w_best, df_notas) # Re-eval dynamic oracle at optimum
match_rate = np.mean(y_pred_model == (get_final_scores(w_best, df_notas) >= 2.95))
print(f"Exactitud: {match_rate:.2%}")


Optimizando con Oráculo Dinámico...

=== PESOS ÓPTIMOS (INTERPOLACIÓN) ===
Modulo 1 (Q+T): 28.6% (Q:14.3%, T:14.3%)
Modulo 2 (Q+T): 28.6% (Q:14.3%, T:14.3%)
Modulo 3 (Q+T): 28.6% (Q:14.3%, T:14.3%)
Examen Final  : 14.3%
Exactitud: 82.35%


## 5. Tabla Detallada de Resultados (Por Módulos)

Mostramos las notas con **1 decimal** y especificamos el peso usado en el encabezado.

In [6]:
w_final = w_best

# 1. Calculate Human/Oracle Decision with FINAL Weights (Dynamic Oracle)
y_oracle_final = get_oracle_decisions_vectorized(w_final, df_notas)
df_notas['Aprueba_Humano'] = y_oracle_final

# 2. Calculate Model Grades
N_final, A, S_local, F_local, m_floor = calcular_nota_interpolacion_vectorizada(w_final, df_notas)

df_res = df_notas.copy()
df_res['Promedio_Clasico_A'] = A
df_res['Nota_Final_N'] = N_final
df_res['Factor_Local_F'] = F_local
df_res['Piso_m'] = m_floor
df_res['Aprueba_Modelo'] = (N_final >= 2.95).astype(int)

# Estados
conditions = [
    (df_res['Aprueba_Humano'] == 1) & (df_res['Aprueba_Modelo'] == 1),
    (df_res['Aprueba_Humano'] == 0) & (df_res['Aprueba_Modelo'] == 0),
    (df_res['Aprueba_Humano'] == 0) & (df_res['Aprueba_Modelo'] == 1),
    (df_res['Aprueba_Humano'] == 1) & (df_res['Aprueba_Modelo'] == 0)
]
choices = ['COINCIDE (Pasa)', 'COINCIDE (Pierde)', 'FALSO POSITIVO', 'FALSO NEGATIVO']
df_res['Estado'] = np.select(conditions, choices, default='ERROR')

cols = ['M1_Q', 'M1_T', 'M2_Q', 'M2_T', 'M3_Q', 'M3_T', 'Examen_Final', 
        'Piso_m', 'Promedio_Clasico_A', 'Factor_Local_F', 'Nota_Final_N', 'Aprueba_Humano', 'Estado']

sample = pd.concat([
    df_res[df_res['Estado'].str.contains('FALSO')].sample(min(5, len(df_res[df_res['Estado'].str.contains('FALSO')]))),
    df_res[df_res['Estado'].str.contains('COINCIDE')].sample(5)
])

def color_rows(s):
    if 'FALSO' in str(s['Estado']):
        return ['background-color: #ffcccc'] * len(s)
    return [''] * len(s)

print("=== REPORTE DETALLADO ESQUEMA INTERPOLACIÓN ===")
# Use try-except for display in case script runs outside notebook
try:
    display(sample[cols].sort_values('Estado').style.apply(color_rows, axis=1).format({
        'Piso_m': '{:.2f}', 'Promedio_Clasico_A': '{:.2f}', 'Factor_Local_F': '{:.2f}', 'Nota_Final_N': '{:.2f}'
    }))
except NameError:
    print(sample[cols].sort_values('Estado'))

# --- ESTADÍSTICAS SOLICITADAS ---
total = len(df_res)
fp_count = (df_res['Estado'] == 'FALSO POSITIVO').sum()
fn_count = (df_res['Estado'] == 'FALSO NEGATIVO').sum()
tn_count = (df_res['Estado'] == 'COINCIDE (Pierde)').sum()

frac_fp = fp_count / total
frac_fn = fn_count / total
total_real_negatives = fp_count + tn_count
fpr = fp_count / total_real_negatives if total_real_negatives > 0 else 0.0

print("\n=== ESTADÍSTICAS DE ERROR ===")
print(f"Fracción de Falsos Positivos (Global): {frac_fp:.2%}")
print(f"Fracción de Falsos Negativos (Global): {frac_fn:.2%}")
print(f"Prob. de ganar sin merecerlo (FPR):    {fpr:.2%}  <-- (FP / Real Negatives)")


=== REPORTE DETALLADO ESQUEMA INTERPOLACIÓN ===


,M1_Q,M1_T,M2_Q,M2_T,M3_Q,M3_T,Examen_Final,Piso_m,Promedio_Clasico_A,Factor_Local_F,Nota_Final_N,Aprueba_Humano,Estado
333,4.240000,1.670000,3.490000,3.600000,4.080000,4.490000,3.360000,2.95,3.56,0.99,3.55,1,COINCIDE (Pasa)
1593,3.670000,4.010000,3.770000,4.010000,2.310000,3.610000,3.780000,2.96,3.59,0.99,3.59,1,COINCIDE (Pasa)
595,4.460000,3.650000,4.320000,4.420000,4.230000,4.330000,2.530000,4.05,3.99,0.86,3.99,1,COINCIDE (Pasa)
1473,3.510000,3.400000,1.080000,1.960000,2.230000,3.350000,3.410000,1.52,2.71,0.56,2.71,0,COINCIDE (Pierde)
2,0.010000,4.030000,0.630000,1.460000,1.830000,3.580000,2.940000,1.04,2.07,0.30,2.07,0,COINCIDE (Pierde)
246,4.020000,4.180000,0.870000,4.520000,2.580000,1.720000,3.780000,2.15,3.10,0.71,2.82,1,FALSO NEGATIVO
154,4.190000,3.430000,3.200000,4.490000,1.970000,2.250000,2.640000,2.11,3.17,0.68,2.83,1,FALSO NEGATIVO
32,4.100000,3.890000,4.510000,4.410000,4.850000,2.220000,1.670000,3.53,3.66,0.59,3.61,0,FALSO POSITIVO
1049,2.980000,3.120000,4.520000,5.000000,3.740000,3.270000,2.250000,3.05,3.55,0.77,3.44,0,FALSO POSITIVO
757,3.210000,4.030000,1.290000,2.700000,3.340000,4.050000,5.000000,1.99,3.37,0.72,2.99,0,FALSO POSITIVO



=== ESTADÍSTICAS DE ERROR ===
Fracción de Falsos Positivos (Global): 15.80%
Fracción de Falsos Negativos (Global): 1.85%
Prob. de ganar sin merecerlo (FPR):    21.97%  <-- (FP / Real Negatives)


In [7]:
3.35*0.47, 2.25

(1.5745, 2.25)